In [1]:
import zhinst.toolkit as tk
import numpy as np

## Connect 2 x HDAWG and 1 x UHFQA

In [2]:
mdc = tk.MultiDeviceConnection()
mdc.setup()

Successfully connected to data server at localhost8004 api version: 6


In [3]:
hd1 = tk.HDAWG("hd1", "dev8030")
mdc.connect_device(hd1)

Successfully connected to device DEV8030 on interface 1GBE


In [4]:
hd2 = tk.HDAWG("hd2", "dev8138")
mdc.connect_device(hd2)

Successfully connected to device DEV8138 on interface 1GBE


In [5]:
qa = tk.UHFQA("qa", "dev2266")
mdc.connect_device(qa)

Successfully connected to device DEV2266 on interface 1GBE


In [6]:
# assign trigger awg and other awgs
trigger = hd1.awgs[0]
awgs = hd1.awgs[1:] + hd2.awgs
readout = qa.awg

In [7]:
# define shared sequence parameters
period = 50e-6
repetitions = 1000

In [8]:
# configure trigger awg with "Trigger" sequence (sets trigger but no waveform)
trigger.set_sequence_params(
    sequence_type="Trigger",
    repetitions=repetitions,
    period=period,
)
trigger.compile()

Current length of queue: 1
Compilation successful
hd1-0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.054004 s


In [107]:
freqs = np.linspace(50e6, 200e6, 7)
amps = np.linspace(1.0, 0.1, 7)

for i, awg in enumerate(awgs):
    awg._trigger_level = 0.25
    awg.set_sequence_params(
        sequence_type="Simple",
        repetitions="inf",
        period=period,
        alignment="End with Trigger",
        trigger_mode="External Trigger",
        trigger_delay=-1e-6*(i+0.5),
    )
    awg.reset_queue()
    l = int(0.5e-6 * 2.4e9)
#     w = (-1)**i * np.ones(l)
    w = amps[i] * np.ones(l)
    awg.queue_waveform(w, -w)
    awg.enable_iq_modulation()
    awg.modulation_freq(freqs[i])
    awg.compile_and_upload_waveforms()

Current length of queue: 1
Compilation successful
hd1-1: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.047972 s
Current length of queue: 1
Compilation successful
hd1-2: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.019993 s
Current length of queue: 1
Compilation successful
hd1-3: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.0099933 s
Current length of queue: 1
Compilation successful
hd2-0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.015973 s
Current length of queue: 1
Compilation successful
hd2-1: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.010975 s
Current length of queue: 1
Compilation successful
hd2-2: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.014992 s
Current length of queue: 1
Compilation successful
hd2-3: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.015007 s


In [108]:
readout.set_sequence_params(
    sequence_type="Simple",
    repetitions="inf",
    period=period,
    alignment="Start with Trigger",
    trigger_mode="External Trigger",
    trigger_delay=-110e-9,
)
readout.reset_queue()
l = int(2e-6 * 1.8e9)
w = np.ones(l)
readout.queue_waveform(w, -w)
readout.compile_and_upload_waveforms()

Current length of queue: 1
Compilation successful
qa-0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.044988 s


In [109]:
for awg in awgs + [readout]:
    awg.outputs(["on", "on"])
    awg.run()

In [110]:
trigger.run()

In [111]:
for awg in awgs + [readout]:
    awg.outputs(["off", "off"])
    awg.stop()
trigger.stop()